In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Normalization, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


2023-03-08 11:43:51.893586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 11:43:51.981404: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-08 11:43:52.356391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/abrar/miniconda3/envs/tf/lib/
2023-03-08 11:43:52.356442: W tensorflow/compiler/

In [3]:

def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    # print(rng)
    X = rng.random(400).reshape(-1,2)
    # print(X)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [4]:
X, Y = load_coffee_data()
print(X.shape)
print(Y.shape)

(200, 2)
(200, 1)


In [5]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51
Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


2023-03-08 11:43:59.440567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-08 11:43:59.440876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-08 11:43:59.441037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-08 11:43:59.441415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-08 11:43:59.441558: I tensorflow/compiler/xla/stream_executo

In [6]:
arr = np.array([[185.32,  12.69], [259.92,  11.87]])
arr_4 = np.tile(arr, (3, 1))
print(arr_4.shape)
print(arr_4)

(6, 2)
[[185.32  12.69]
 [259.92  11.87]
 [185.32  12.69]
 [259.92  11.87]
 [185.32  12.69]
 [259.92  11.87]]


In [7]:
## Increasing training data to decrease training iterations(epochs)
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape)   

(200000, 2) (200000, 1)


In [8]:
tf.random.set_seed(1234) # Needed for getting similar output everytime the code is run
model = Sequential([
    Input(shape=(2,)),
    Dense(units=3, activation='relu', name='layer_1'),
    Dense(units=1, activation='sigmoid', name='layer_2')
])


In [9]:
model.summary() # Checking the model summary

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 3)                 9         
                                                                 
 layer_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [10]:
W1, B1 = model.get_layer("layer_1").get_weights()
print(f"W1:\n{W1}\nB1:\n{B1}")
W2, B2 = model.get_layer("layer_1").get_weights()
print(f"W2:\n{W2}\nB2:\n{B2}")

W1:
[[ 0.29121268  0.59385514  0.9740267 ]
 [-0.8101201  -0.4449491  -0.831807  ]]
B1:
[0. 0. 0.]
W2:
[[ 0.29121268  0.59385514  0.9740267 ]
 [-0.8101201  -0.4449491  -0.831807  ]]
B2:
[0. 0. 0.]


In [11]:
# Compiling model
model.compile(
    loss=BinaryCrossentropy(),
    optimizer= Adam(learning_rate=0.01)
)

# Training Model
model.fit(Xt, Yt, epochs=10)

Epoch 1/10


2023-03-08 11:44:13.915341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-08 11:44:13.916840: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f7522587c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-08 11:44:13.916855: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-03-08 11:44:13.920266: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-08 11:44:14.012551: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6250/6250 [==============================] - 7s 976us/step - loss: 0.1120
Epoch 2/10
6250/6250 [==============================] - 6s 963us/step - loss: 0.0260
Epoch 3/10
6250/6250 [==============================] - 6s 964us/step - loss: 0.0148
Epoch 4/10
6250/6250 [==============================] - 6s 967us/step - loss: 0.0095
Epoch 5/10
6250/6250 [==============================] - 6s 967us/step - loss: 0.0063
Epoch 6/10
6250/6250 [==============================] - 6s 964us/step - loss: 0.0044
Epoch 7/10
6250/6250 [==============================] - 6s 963us/step - loss: 0.0032
Epoch 8/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0023
Epoch 9/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0019
Epoch 10/10
6250/6250 [==============================] - 6s 987us/step - loss: 0.0012


In [12]:
W1, B1 = model.get_layer("layer_1").get_weights()
print(f"W1:\n{W1}\nB1:\n{B1}")
W2, B2 = model.get_layer("layer_1").get_weights()
print(f"W2:\n{W2}\nB2:\n{B2}")

W1:
[[ 7.843036    8.21372     8.875285  ]
 [ 0.43905523  6.9482327  -9.652563  ]]
B1:
[ 9.380385   1.9992826 -2.4938345]
W2:
[[ 7.843036    8.21372     8.875285  ]
 [ 0.43905523  6.9482327  -9.652563  ]]
B2:
[ 9.380385   1.9992826 -2.4938345]


In [13]:
## Testing the model
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 45ms/step
predictions = 
 [[1.]
 [0.]]
